In [1]:
# Importando as bibliotecas necessarias
import numpy as np
import pandas as pd
import os

In [2]:
# Importando os arquivos txt com as informações
pasta = os.getcwd() + "\\Dados TG\\Data Info"

bds_info = pd.read_csv(pasta + "\\BDSinfo.txt", sep="	", header=0)
pds_info = pd.read_csv(pasta + "\\PDSinfo.txt", sep="	", header=0)

In [3]:
print(len(list(bds_info)))
#print(list(bds_info))

64


In [4]:
print(len(list(pds_info)))
# Mudar o nome da coluna para igualar ao do banco BDS
pds_info.columns = pds_info.columns.str.replace('Mass','Weight')
pds_info["Height"] = pds_info["Height"]*100
#print(list(pds_info))

29


In [5]:
rotulos_iguais = set(bds_info) & set(pds_info)
print(len(rotulos_iguais))
print(rotulos_iguais)

25
{'Falls12m', 'Ystudy', 'FootLen', 'Disability', 'Age', 'Height', 'Footwear', 'Nationality', 'Disability2', 'Subject', 'SkinColor', 'Weight', 'AgeGroup', 'Nmedication', 'Illness2', 'Trial', 'BMI', 'Surface', 'Medication', 'Illness', 'Vision', 'Date', 'Gender', 'Ortho-Prosthesis2', 'Ortho-Prosthesis'}


# Juntados os dois Databases BDS e PDS

In [6]:
# Juntando os dois databases em um apenas com os rótulos em comum
info = pd.concat([bds_info, pds_info], join="inner")
#print(info)

In [7]:
info.shape

(2518, 25)

In [8]:
def junta_dupli(df, x):
    #print(len(list(df.columns)))
    #print(df[x].shape)
    df[x + "_total"] = df[x].sum(axis=1)
    df = df.drop([x], 1)
    #print(len(list(df.columns)))
    return df

# Tratando a coluna "Medication"

In [9]:
info = pd.concat([bds_info, pds_info], join="inner")

# Padroniza os separadores como virgula
info["Medication"] = info["Medication"].str.replace('+',',')
info["Medication"] = info["Medication"].str.replace('/',',')

# Padroniza diferentes formatos de escrita para o mesmo medicamento
info["Medication"] = info["Medication"].str.replace('-',' ')
info["Medication"] = info["Medication"].str.replace('2','')
info["Medication"] = info["Medication"].str.replace('(','')
info["Medication"] = info["Medication"].str.replace(')','')

# Retira duplicatas
info["Medication"] = info["Medication"].str.replace('Bisphosphonates','Bisphosphonate')
info["Medication"] = info["Medication"].str.replace('Long acting beta agonists','Long acting beta agonist')
info["Medication"] = info["Medication"].str.replace('Selective serotonin and norepinephrine reuptake inhibitor','Selective serotonin reuptake inhibitor, Selective norepinephrine reuptake inhibitor')
info["Medication"] = info["Medication"].str.replace('HMG coenzyme A reductase inhibitor','HMG CoA reductase inhibitor')
info["Medication"] = info["Medication"].str.replace('Nonsteroidal antiinflammatory drug','Nonsteroidal antiinflammatory')

# Retira os espaços a mais
info["Medication"] = info["Medication"].str.replace(' ','')

medication = info["Medication"].str.get_dummies(sep=',')
                        
print(len(list(medication.columns)))
list(medication.columns)

68


['5alphareductaseinhibitor',
 'Alphablocker',
 'Analgesic',
 'AngiotensinIIreceptorantagonist',
 'Angiotensinconvertingenzymeinhibitor',
 'Antiarrhythmic',
 'Antibiotic',
 'Anticholinergic',
 'Anticonvulsant',
 'Antihistamine',
 'Antiinflammatory',
 'Antimalarial',
 'Antimuscarinic',
 'Antipyretic',
 'Antithyroid',
 'Antivertigo',
 'Benzodiazepine',
 'Betablocker',
 'Biguanide',
 'Bisphosphonate',
 'Bloodcirculation',
 'Bronchodilator',
 'Calciumchannelblocker',
 'Centralnervoussystemagent',
 'Cholinesteraseinhibitor',
 'Conjugatedestrogen',
 'Corticosteroid',
 'DOPAdecarboxylaseinhibitor',
 'Diabetesmellitus',
 'Dipeptidylpeptidase4inhibitor',
 'Diuretic',
 'Dopamineagonist',
 'Dopamineantagonist',
 'Essentialaminoacid',
 'Fluoroquinolone',
 'HMGCoAreductaseinhibitor',
 'Hblocker',
 'Herbalsupplement',
 'Homeopathic',
 'Hypercholesterolemia',
 'Hypertension',
 'Insulin',
 'Kinaseinhibitor',
 'Kneecartilagereplacement',
 'Longactingbetaagonist',
 'Meglitinide',
 'Musclerelaxant',
 'No'

# Tratando a coluna "Illness2"

Tomar cuidado, nesse caso substituiu a linha inteira de illness que começava com "Tendinitis", inclusive as outras doenças após a vírgula:

info["Illness2"].replace('Tendinitis (.*)', 'Tendinitis', regex=True, inplace=True)

In [10]:
info = pd.concat([bds_info, pds_info], join="inner")
illness = info["Illness2"].str.get_dummies(sep=',')
illness.columns = illness.columns.str.replace('Heart problem','Heart disease')
illness.columns = illness.columns.str.replace('Osteeopenia','Osteopenia')
illness.columns = illness.columns.str.replace('Tendinitis (.*)', 'Tendinitis')
illness.columns = illness.columns.str.replace('Arthrosis (.*)', 'Arthrosis')
illness.columns = illness.columns.str.replace('(.*) Herniated (.*)', 'Herniated lumbar disc')
illness.columns = illness.columns.str.replace('Herniated (.*)', 'Herniated lumbar disc')
illness.columns = illness.columns.str.replace('(.*) Hypercholesterolemia','Hypercholesterolemia')
illness.columns = illness.columns.str.replace('Skin allergy','Allergy')
illness.columns = illness.columns.str.strip()
illness = illness.drop(["No"], 1)

# Juntando as colunas iguais e excluindo os repetidos

illness = junta_dupli(illness, "Heart disease")
illness = junta_dupli(illness, "Osteopenia")
illness = junta_dupli(illness, "Tendinitis")
illness = junta_dupli(illness, "Arthrosis")
illness = junta_dupli(illness, "Herniated lumbar disc")
illness = junta_dupli(illness, "Hypercholesterolemia")
illness = junta_dupli(illness, "Allergy")
illness = junta_dupli(illness, "Anxiety")
illness = junta_dupli(illness, "Asthma")
illness = junta_dupli(illness, "Chodromalacia of the knees")
illness = junta_dupli(illness, "Diabetes mellitus")
illness = junta_dupli(illness, "Fibromyalgia")
illness = junta_dupli(illness, "Hypertension")
illness = junta_dupli(illness, "Hyperthyroidism")
illness = junta_dupli(illness, "Hypoglycemia")
illness = junta_dupli(illness, "Hypothyroidism")
illness = junta_dupli(illness, "Labyrinthitis")
illness = junta_dupli(illness, "Osteoporosis")
illness = junta_dupli(illness, "Rhinitis")
illness = junta_dupli(illness, "Scoliosis")
                        
print(len(list(illness.columns)))
list(illness.columns)

77


['Arthritis',
 'Breathlessness',
 'Bronchitis',
 'Bursitis',
 'Conjunctivitis',
 'Dermatitis',
 'Epilepsy',
 'Esophagitis',
 'Essential tremor',
 'Fatty liver',
 'Gastritis',
 'Gastroesophageal reflux',
 'Glaucoma',
 'Gout',
 'Hiatus hernia',
 'Hypertriglyceridemia',
 'Hyperuricemia',
 'Inflammation of the tibial nerve',
 'Keratoconus',
 'Kidney failure',
 'Kidney stones',
 'Ligament problems',
 'Lumbar tumor',
 'Lymphedema of the lower members',
 'Migraine',
 'Poliomyelitis',
 'Polycystic kidney disease',
 'Progressive breast cancer',
 'Prostate disease',
 'Prostatitis',
 'Rheumatism',
 'Sickle cell anemia',
 'Sinusitis',
 'Skin cancer',
 'Skin disease',
 'Spine problem',
 'Stroke',
 'Varicose veins of the legs',
 'Vascular leakage of the eyes',
 'Anemia',
 'Arrhythmia',
 'Ascending colon cancer',
 'Deep vein thrombosis',
 'Depression',
 'Endometriosis',
 'Hashimoto disease',
 'Heel spurs',
 'Hepatitis B',
 'Intestine disorder',
 'Melanoma',
 "Parkinson's disease",
 'Prostatic hyperpl

# Tratando a coluna "Disability2"

In [11]:
info = pd.concat([bds_info, pds_info], join="inner")
disability = info["Disability2"].str.get_dummies(sep=',')
disability.columns = disability.columns.str.strip()
disability = disability.drop(["No"], 1)
disability.columns = disability.columns.str.replace('Physical (.*)', 'Physical')
disability.columns = disability.columns.str.replace('Visual (.*)', 'Visual')
disability.columns = disability.columns.str.replace('Hearing (.*)', 'Hearing')

disability = junta_dupli(disability, "Physical")
disability = junta_dupli(disability, "Visual")
disability = junta_dupli(disability, "Hearing")

print(len(list(disability.columns)))
list(disability.columns)

4


['Intellectual', 'Physical_total', 'Visual_total', 'Hearing_total']

# Tratando a coluna "Footwear"

In [12]:
info = pd.concat([bds_info, pds_info], join="inner")
footwear = info["Footwear"].str.get_dummies(sep=',')
footwear.columns = footwear.columns.str.strip()

footwear.columns = footwear.columns.str.replace('(.*) shoes with silicone insole', 'Shoes with silicone insole')
footwear.columns = footwear.columns.str.replace('Orthopedic shoes', 'Orthopedic Shoes')
footwear.columns = footwear.columns.str.replace('(.*) shoes', 'Normal Shoes')
footwear.columns = footwear.columns.str.replace('Tennis with zero drop', 'Normal Shoes')
footwear.columns = footwear.columns.str.replace('Sandal', 'Sandal_FlipFlop')
footwear.columns = footwear.columns.str.replace('Flip-Flops', 'Sandal_FlipFlop')

footwear = junta_dupli(footwear, "Boots")
footwear = junta_dupli(footwear, "Shoes with silicone insole")
footwear = junta_dupli(footwear, "Normal Shoes")
footwear = junta_dupli(footwear, "Sandal_FlipFlop")

print(len(list(footwear.columns)))
list(footwear.columns)

5


['Orthopedic Shoes',
 'Boots_total',
 'Shoes with silicone insole_total',
 'Normal Shoes_total',
 'Sandal_FlipFlop_total']

# Tratando a coluna "Ortho-Prosthesis2"

In [13]:
info["Ortho-Prosthesis2"]

info = pd.concat([bds_info, pds_info], join="inner")
info["Ortho-Prosthesis2"] = info["Ortho-Prosthesis2"].str.replace('1 intramedullary rod and 3 screws in the left leg', '1 intramedullary rod, 3 screws in the left leg')
ortho = info["Ortho-Prosthesis2"].str.get_dummies(sep=',')
ortho.columns = ortho.columns.str.strip()
ortho = ortho.drop(["No"], 1)

ortho.columns = ortho.columns.str.replace('Dental (.*)', 'Dental')
ortho.columns = ortho.columns.str.replace('Bridge teeh', 'Dental')
ortho.columns = ortho.columns.str.replace('Denture', 'Dental')
ortho.columns = ortho.columns.str.replace('Protese dentaria', 'Dental')
ortho.columns = ortho.columns.str.replace('Crutch', 'Support')
ortho.columns = ortho.columns.str.replace('Cane', 'Support')
ortho.columns = ortho.columns.str.replace('Hearing aid', 'Hearing')
ortho.columns = ortho.columns.str.replace('1 screw in the right toe', 'Internal fixation in the toe_foot')
ortho.columns = ortho.columns.str.replace('Screws in the left foot', 'Internal fixation in the toe_foot')
ortho.columns = ortho.columns.str.replace('2 screws in the right foot', 'Internal fixation in the toe_foot')
ortho.columns = ortho.columns.str.replace('2 screws in the right thigh', 'Internal fixation in the thigh')
ortho.columns = ortho.columns.str.replace('3 screws in the left leg', 'Internal fixation in the leg')
ortho.columns = ortho.columns.str.replace('1 intramedullary rod', 'Internal fixation in the medulla')
ortho.columns = ortho.columns.str.replace('Anchor in the shoulders', 'Internal fixation in the shoulder')
ortho.columns = ortho.columns.str.replace('16 screws and 4 plates in the left knee', 'Internal fixation in the knee')
ortho.columns = ortho.columns.str.replace('(.*) knee (.*)', 'Orthosis in the knee')



ortho = junta_dupli(ortho, "Dental")
ortho = junta_dupli(ortho, "Support")
ortho = junta_dupli(ortho, "Hearing")
ortho = junta_dupli(ortho, "Internal fixation in the toe_foot")
ortho = junta_dupli(ortho, "Orthosis in the knee")

print(len(list(ortho.columns)))
list(ortho.columns)

17


['Internal fixation in the thigh',
 'Internal fixation in the leg',
 'Internal fixation in the shoulder',
 'Breast implant',
 'By pass',
 'Clips in the vesicle',
 'Corrective lens',
 'PaceMaker',
 'Internal fixation in the medulla',
 'Internal fixation in the knee',
 'Corrective lens',
 'Prosthesis aortic valve',
 'Dental_total',
 'Support_total',
 'Hearing_total',
 'Internal fixation in the toe_foot_total',
 'Orthosis in the knee_total']